In [35]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import pickle
import string

# Input the CIK list for the fund family you are looking at

In [36]:
fund_name = 'vanguard'


CIK_LIST = ['106444', '36405', '752177', '225997', '734383', '932471', '794105', '826473', '1273878', '857489', '1004655', '836906', '857490', '821404', '788599', '862341', '788606', '891190', '791107', '34066', '1070414', '105544', '105563', '68138', '52848', '799127', '736054', '106830', '313850', '107606']




In [37]:
cur_wd = os.getcwd()
if 'trunk' in cur_wd:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output')
else:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output','working')
    
cached_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'cached_data')
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print('data directory')
print(cur_wd)
print('output directory')
print(output_directory)

print('cached data')
print(cached_wd)


data directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\data
output directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\output\working
cached data
C:\Users\jjanko\Desktop\wei_mutual_fund_project\cached_data


# Helper Functions for Analysis

In [38]:
def get_cleaned(lines):
    cleaned = []
    for i in lines:
        #gets rid of tags
        cleaned_i = re.sub('<[^>]*>', ' ', i)
        #gets rid of &nbsp; and replaces with space
        cleaned_i = re.sub('&nbsp;', ' ', cleaned_i)
        #re.sub("<.*?>","",st)
        if not re.match(r'^\s*$', cleaned_i):
            cleaned.append(cleaned_i)
    return cleaned

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))


def remove_junk(addline):
    addline = re.sub('<[^>]*>', ' ', addline)
    addline = re.sub('&nbsp;', ' ', addline)
    addine = re.sub('&#174;', ' ', addline)
    addline = re.sub('&#8480;', ' ', addline)
    addline = re.sub(r'&reg;',' ',addline)
    addline = re.sub(r'\n', ' ', addline)
    addline = re.sub(r'\t', ' ', addline)
    return addline


In [39]:
with open(os.path.join(cached_wd, 'errors_dict.pickle'), 'rb') as handle:
    errors = pickle.load(handle)
    
print(errors)

        

{'867297': ['0001193125-04-202597.txt', '0001193125-05-115532.txt', '0001193125-05-232633.txt'], '1091439': ['0000950152-04-008622.txt', '0000950152-05-004867.txt', '0000950152-05-009599.txt'], '1301708': ['0000950152-05-009600.txt'], '1423227': [], '1098605': ['0000950136-05-006141.txt', '0000950136-04-003174.txt', '0000950136-05-001871.txt'], '1378240': ['0001104659-07-029038.txt', '0001104659-07-077264.txt', '0001104659-08-027201.txt', '0001104659-08-064591.txt'], '726735': ['0000950129-04-009378.txt', '0000950129-05-005810.txt', '0000950129-05-011455.txt'], '844779': ['0001193125-05-039836.txt', '0001193125-05-175146.txt', '0001193125-05-175147.txt', '0001193125-05-175620.txt'], '1398078': [], '774013': ['0001193805-05-000319.txt', '0001193805-05-001781.txt', '0001193805-05-002628.txt', '0001193805-05-002636.txt', '0001193805-05-002638.txt'], '1062806': ['0000900092-04-000310.txt', '0000900092-05-000103.txt', '0000900092-05-000135.txt', '0000900092-05-000453.txt', '0001193805-04-00

In [40]:
weblink = {}
txt = []

for CIK in CIK_LIST:
    weblink[CIK]  = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        txt.append(i)
        weblink[CIK].append(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK)+r"/"  + i)

get panel 2

In [41]:
with open(os.path.join(cached_wd, 'panel2.pickle'), 'rb') as handle:
    panel2 = pickle.load(handle)
    


In [42]:
panel2 = panel2.loc[panel2['file_read'].isin(txt)]

In [43]:
print(panel2)
panel2.to_csv(os.path.join(output_directory, fund_name + '_panel2.csv'), sep = ',')

   CLASS-CONTRACT-ID   CLASS-CONTRACT-NAME CLASS-CONTRACT-TICKER-SYMBOL  \
0         C000007087       Investor Shares                        VFIIX   
1         C000007088        Admiral Shares                        VFIJX   
2         C000007089       Investor Shares                        VFISX   
3         C000007090        Admiral Shares                        VFIRX   
4         C000007092       Investor Shares                        VWEHX   
5         C000007093        Admiral Shares                        VWEAX   
6         C000007094       Investor Shares                        VIPSX   
7         C000007095        Admiral Shares                        VAIPX   
8         C000007096  Institutional Shares                        VIPIX   
9         C000007097       Investor Shares                        VFICX   
10        C000007098        Admiral Shares                        VFIDX   
11        C000007099       Investor Shares                        VFITX   
12        C000007100     

## Test on One FILE

In [44]:

CIK  ='857489'
text =  '0000932471-08-001886.txt'


In [45]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()
print(names)

['Vanguard Emerging Markets Stock Index Fund'
 'Vanguard European Stock Index Fund' 'Vanguard Pacific Stock Index Fund'
 'Vanguard FTSE All-World ex-US Index Fund'
 'Vanguard Total World Stock Index Fund']


In [46]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
html_table = urlopen(url).read()
soup = BeautifulSoup(html_table, "html.parser")
tags = soup.find_all(['table', 'p'])

In [47]:
print(url)

https://www.sec.gov/Archives/edgar/data/857489/000093247108001886/0000932471-08-001886.txt


In [48]:
tables = []

for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
            print(fundname)
            
    try:
        df = pd.read_html(str(i), flavor="bs4")
        for j in df:
            if len(j.index) > 0 :
                tables.append(j)
    except:
        pass

Vanguard Pacific Stock Index Fund
Vanguard European Stock Index Fund
Vanguard Emerging Markets Stock Index Fund


In [49]:
tables[10]

,0,1,2,3
0,NaN,Nomura Holdings Inc.,7320200,105640
1,NaN,"JFE Holdings, Inc.",2160900,105027
2,NaN,"Shin-Etsu Chemical Co., Ltd.",1698400,103912
3,NaN,"Seven and I Holdings Co., Ltd.",3363540,102864
4,NaN,Hitachi Ltd.,13936000,100460
5,NaN,Komatsu Ltd.,3718968,92358
6,NaN,Astellas Pharma Inc.,2039798,88445
7,NaN,"Japan Tobacco, Inc.",18620,86791
8,NaN,"Daiichi Sankyo Co., Ltd.",2888936,86253
9,NaN,Toshiba Corp.,12724000,82612


In [50]:
def get_legend(df):
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    legend = pd.DataFrame(columns = ['code', 'symbol', 'restricted'])
    if df[df.columns[1]].apply(lambda x: str(x).lower()).str.contains('non-income').any():
        for count, row in df.iterrows():
            keys = str(row[0])
            code = str(row[1])
            if (keys.strip() != code.strip()) and (code.strip() != ''):
                if len(keys) == 1:
                    add = pd.DataFrame([code, keys, '']).T
                    add.columns = ['code', 'symbol', 'restricted']
                    if len(legend.index) == 0:
                        legend = add.copy()
                    else:
                        legend = pd.concat([legend, add], axis = 0,ignore_index = True, sort = False)
    return legend

### check legend 

In [51]:
try:
    df = tables[158]
    legend = get_legend(df)
    print(legend)
except: 
    print("you are reading the wrong table")

you are reading the wrong table


In [52]:
def check_level_columns(df):
    check_found = True
    for col in df.columns:
        check_name = df[col][df[col].apply(lambda x: re.sub(r'\s', '', str(x)).lower()).str.contains('level1')]
        if len(check_name) > 0:
            check_found = False
    return check_found

In [86]:
def get_holdings(df, headername):
    cleaned_df = pd.DataFrame()
    df = df.dropna(axis = 0, how = 'all')
    check_levels = check_level_columns(df)
    
    if len(df.columns) >= 4 and check_levels:
        df = df.dropna(axis = 0, subset = [df.columns[1]])
        df = df.dropna(thresh = 3, axis = 0)
        non_column_name = df.drop(df.columns[1], axis = 1)
        non_column_name = non_column_name.drop(df.columns[0], axis = 1)
        for count, row in df.iterrows():
            name = str(row[1]).lower().strip()
            key = str(row[0]).strip()
            if key == 'nan':
                key = ''
            substringFound = False
            for i in filter_list:
                if i in name:
                    
                
            if name[-1] != r'%' and '%)' not in name and '(cost' not in name and substringFound is False:
                if ':' in name:
                    headername = name
                if name.split(' ')[0] == 'Series' or name[0].isdigit():
                    name = headername + ' ' + name

                add = pd.DataFrame(non_column_name.loc[count]).dropna()
                add = add[~add.duplicated()]
                if len(add) == 2:
                    add = add.transpose().dropna()
                    add.columns = ['holdings shares', 'holdings value']
                    add['holdings name'] = name
                    add['key'] = key

                    cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                    cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))

    if len(cleaned_df) == 0:

        if len(df.columns) >= 4 and check_levels:
            df = df.dropna(axis = 0, subset = [df.columns[1]])
            cleaned_df = pd.DataFrame()
            df = df.dropna(thresh = 3, axis = 0)
            non_column_name = df.drop(df.columns[0], axis = 1)
            for count, row in df.iterrows():
                name = str(row[0]).strip()
                key = str(row[0])[0].lower()
                if key == 'nan':
                    key = ''

                if name[-1] != r'%' and '%)' not in name and '(cost' not in name.lower() \
                and 'total' not in name.lower():
                    if ':' in name:
                        headername = name
                    if name.split(' ')[0] == 'Series' or name[0].isdigit():
                        name = headername + ' ' + name

                    add = pd.DataFrame(non_column_name.loc[count]).dropna()
                    add = add[~add.duplicated()]
                    if len(add) == 2:
                        add = add.transpose().dropna()
                        add.columns = ['holdings shares', 'holdings value']
                        add['holdings name'] = name
                        add['key'] = re.sub(r'[a-zA-Z]', '', key)

                        cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                        cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))


    return cleaned_df, headername
    

In [54]:
headername = ''
df,headername = get_holdings(tables[10], headername)
print(df)

    holdings shares  holdings value  \
0           7320200          105640   
1           2160900          105027   
2           1698400          103912   
3           3363540          102864   
4          13936000          100460   
5           3718968           92358   
6           2039798           88445   
7             18620           86791   
8           2888936           86253   
9          12724000           82612   
10          7996000           78805   
11          3465557           78405   
12         15908000           76530   
13          3186400           74299   
14          9351720           71992   
15            12063           69104   
16             6489           66000   
17          2739600           65800   
18          2022766           63582   
19           792800           62958   
20          4656756           62859   
21          6230000           61911   
22          4742000           61410   
23         13261000           58469   
24           672900      

# check holdings

In [55]:
df = tables[10].dropna(axis = 0, how = 'all')
cleaned_df = pd.DataFrame()
if len(df.columns) >= 4:
    df = df.dropna(axis = 0, subset = [df.columns[1]])

    df = df.dropna(thresh = 3, axis = 0)
    non_column_name = df.drop(df.columns[1], axis = 1)
    non_column_name = non_column_name.drop(df.columns[0], axis = 1)
    for count, row in df.iterrows():
        name = str(row[1])
        key = str(row[0])
        if key == 'nan':
            key = ''

        if name[-1] != r'%':
            if ':' in name:
                headername = name
            if name.split(' ')[0] == 'Series' or name[0].isdigit():
                name = headername + ' ' + name

            add = pd.DataFrame(non_column_name.loc[count]).dropna()
            add = add[~add.duplicated()]
            if len(add) == 2:
                add = add.transpose().dropna()
                add.columns = ['holdings shares', 'holdings value']
                add['holdings name'] = name
                add['key'] = key

                cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))


if len(cleaned_df) == 0:
    
    df = df.dropna(axis = 0, subset = [df.columns[1]])
    cleaned_df = pd.DataFrame()
    df = df.dropna(thresh = 3, axis = 0)
    non_column_name = df.drop(df.columns[0], axis = 1)
    for count, row in df.iterrows():
        name = str(row[0])
        key = str(row[0])[0].lower()
        if key == 'nan':
            key = ''

        if name[-1] != r'%':
            if ':' in name:
                headername = name
            if name.split(' ')[0] == 'Series' or name[0].isdigit():
                name = headername + ' ' + name

            add = pd.DataFrame(non_column_name.loc[count]).dropna()
            add = add[~add.duplicated()]
            if len(add) == 2:
                add = add.transpose().dropna()
                add.columns = ['holdings shares', 'holdings value']
                add['holdings name'] = name
                add['key'] = re.sub(r'[a-zA-Z]', '', key)

                cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
print(cleaned_df)

    holdings shares  holdings value  \
0           7320200          105640   
1           2160900          105027   
2           1698400          103912   
3           3363540          102864   
4          13936000          100460   
5           3718968           92358   
6           2039798           88445   
7             18620           86791   
8           2888936           86253   
9          12724000           82612   
10          7996000           78805   
11          3465557           78405   
12         15908000           76530   
13          3186400           74299   
14          9351720           71992   
15            12063           69104   
16             6489           66000   
17          2739600           65800   
18          2022766           63582   
19           792800           62958   
20          4656756           62859   
21          6230000           61911   
22          4742000           61410   
23         13261000           58469   
24           672900      

In [56]:
    df = df.dropna(axis = 0, subset = [df.columns[1]])
    cleaned_df = pd.DataFrame()
    df = df.dropna(thresh = 3, axis = 0)
    non_column_name = df.drop(df.columns[1], axis = 1)
    non_column_name = non_column_name.drop(df.columns[0], axis = 1)

# no acquisition data for vanguard

# Run both legend and holdings scraper on test file

In [57]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()
    
headername = ''
tables = []
holdings = pd.DataFrame()
legend = pd.DataFrame()
fundname = ''
identifiercount = 0
count = 0

for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
            print(fundname)
            
    if r'<p' in str(i):
        s = re.sub('<.*?>', '', str(i)).replace('\xa0','')
        a = s.split(' ')
        if (len(a[0]) == 1) and (len(a) > 1) and (not a[0].isalpha()):
            if 'variation margin' not in s.lower() and 'payment received' not in s.lower() and 'applicable—purchases' not in s.lower() and 'pursuant to a' not in s.lower():
                code = str(s[2:])
                keys = str(a[0])
                if (keys.strip() != code.strip()) and (code.strip() != ''):
                    identifier = fundname

                    add_l = pd.DataFrame([code, keys, identifier, '']).T
                    add_l.columns = ['code', 'symbol', 'identifier', 'restricted']
                    legend = legend.append(add_l, ignore_index = True, sort = False)            
    try:
        if fundname != '':
            df = pd.read_html(str(i), flavor="bs4")
            for t in df:
                count = count +1
                tables.append(t)
                add_legend = get_legend(t)
                if len(add_legend.index) > 0:
                    add_legend.loc[:, 'identifier'] = fundname
                    if len(legend.index) > 0:
                        legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
                    else:
                        legend  = add_legend.copy()

                cleaned_df, headername = get_holdings(t, headername)
                if len(cleaned_df.index) > 1:
                    cleaned_df.loc[:, 'fund name'] = fundname
                    cleaned_df.loc[:, 'identifier'] = fundname
                    if len(holdings.index) > 0:
                        holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
                    else:
                        holdings = cleaned_df.copy()
    except: 
        pass


Vanguard Pacific Stock Index Fund
Vanguard European Stock Index Fund
Vanguard Emerging Markets Stock Index Fund


In [58]:
legend

,code,symbol,restricted,identifier
0,Part of security position is on loan to broker...,^,,Vanguard Pacific Stock Index Fund
1,Non-income-producing security.,*,,Vanguard Pacific Stock Index Fund
2,The issuer operates under a congressional char...,1,,Vanguard Pacific Stock Index Fund
3,"Securities with a value of $9,983,000 have bee...",2,,Vanguard Pacific Stock Index Fund
4,Affiliated money market fund available only to...,3,,Vanguard Pacific Stock Index Fund
5,Non-income-producing security.,*,,Vanguard European Stock Index Fund
6,Part of security position is on loan to broker...,^,,Vanguard European Stock Index Fund
7,The issuer operates under a congressional char...,1,,Vanguard European Stock Index Fund
8,"Securities with a value of $21,974,000 have be...",2,,Vanguard European Stock Index Fund
9,Affiliated money market fund available only to...,3,,Vanguard European Stock Index Fund


In [59]:
print(holdings['identifier'].unique())
print(holdings)

['Vanguard Pacific Stock Index Fund' 'Vanguard European Stock Index Fund'
 'Vanguard Emerging Markets Stock Index Fund']
                                     holdings shares holdings value  \
0                                           13884860         518126   
1                                            5444483         201893   
2                                            7770790         156007   
3                                            6736317         154638   
4                                            1182224         138388   
5                                            7947234         120314   
6                                            5024978         118503   
7                                            7255852         109506   
8                                            1993446         100080   
9                                            2616034          83878   
10                                           3666542          77400   
11                         

In [60]:
holdings['value multiplier'] = ''
holdings['textfile'] = text
holdings['CIK'] = CIK
holdings['date_filed'] = '' 
holdings['company conformed name'] = ''
holdings['reporting_date'] = ''

try:
    date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
    conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
    reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
except IndexError:
    date_filed = ''
    conformed_name = ''

holdings.loc[:, 'date_filed'] = date_filed
holdings.loc[:, 'company conformed name'] = conformed_name
holdings.loc[:, 'reporting_date'] = reporting_date

holdings.index = pd.RangeIndex(len(holdings.index))
print(holdings)

for z in weblink[CIK]:
    if text in z:
        matching_link = z

holdings['weblink'] = matching_link
legend['weblink'] = matching_link
legend['textfile'] = text

                                     holdings shares holdings value  \
0                                           13884860         518126   
1                                            5444483         201893   
2                                            7770790         156007   
3                                            6736317         154638   
4                                            1182224         138388   
5                                            7947234         120314   
6                                            5024978         118503   
7                                            7255852         109506   
8                                            1993446         100080   
9                                            2616034          83878   
10                                           3666542          77400   
11                                          18017923          76061   
12                                           2276826          73618   
13    

In [61]:
print(holdings)
print(legend)

                                     holdings shares holdings value  \
0                                           13884860         518126   
1                                            5444483         201893   
2                                            7770790         156007   
3                                            6736317         154638   
4                                            1182224         138388   
5                                            7947234         120314   
6                                            5024978         118503   
7                                            7255852         109506   
8                                            1993446         100080   
9                                            2616034          83878   
10                                           3666542          77400   
11                                          18017923          76061   
12                                           2276826          73618   
13    

In [62]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") |legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'



                                                 code symbol restricted  \
12  Security exempt from registration under Rule 1...      1        yes   
17  Security exempt from registration under Rule 1...      1        yes   

                                    identifier  \
12  Vanguard Emerging Markets Stock Index Fund   
17  Vanguard Emerging Markets Stock Index Fund   

                                              weblink  \
12  https://www.sec.gov/Archives/edgar/data/857489...   
17  https://www.sec.gov/Archives/edgar/data/857489...   

                    textfile  
12  0000932471-08-001886.txt  
17  0000932471-08-001886.txt  


In [63]:
#get the restricted legend and restricted securities

In [64]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") | legend['code'].str.contains("private placement transaction"), 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    holdings.loc[(holdings['identifier'] == row['identifier']) & (holdings['textfile'] == row.textfile) \
                   & (holdings['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    
restricted_holdings = holdings.loc[(holdings['restricted'] == 'yes')]
print(restricted_holdings)

                                                 code symbol restricted  \
12  Security exempt from registration under Rule 1...      1        yes   
17  Security exempt from registration under Rule 1...      1        yes   

                                    identifier  \
12  Vanguard Emerging Markets Stock Index Fund   
17  Vanguard Emerging Markets Stock Index Fund   

                                              weblink  \
12  https://www.sec.gov/Archives/edgar/data/857489...   
17  https://www.sec.gov/Archives/edgar/data/857489...   

                    textfile  
12  0000932471-08-001886.txt  
17  0000932471-08-001886.txt  
     holdings shares holdings value  \
1626          286260          79191   
3448           50312            332   
3451           22844            150   
3452           28033            100   
3453           11141             70   
3454            9790             65   
3455            4303             60   
3456            8503             50   
3457   

In [65]:
#get the restricted holdings

In [66]:
def getpanel1(CIK, text, panel2):
    
    url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
    html_table = urlopen(url).read()
    soup = BeautifulSoup(html_table, "html.parser")
    tags = soup.find_all(['table', 'p'])
    names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
    names = names.unique()

    headername = ''
    tables = []
    holdings = pd.DataFrame()
    legend = pd.DataFrame()
    fundname = ''
    identifiercount = 0
    count = 0
    

    for i in tags:
        for n in names:
            if n.lower() in str(i).lower().replace('\n',' '):
                fundname = n

        if r'<p' in str(i):
            s = re.sub('<.*?>', '', str(i)).replace('\xa0','')
            a = s.split(' ')
            if (len(a[0]) == 1) and (len(a) > 1) and (not a[0].isalpha()):
                if 'variation margin' not in s.lower() and 'payment received' not in s.lower() and \
                'applicable—purchases' not in s.lower() and 'pursuant to a' not in s.lower():
                    code = str(s[2:])
                    keys = str(a[0])
                    if (keys.strip() != code.strip()) and (code.strip() != ''):
                        identifier = fundname

                        add_l = pd.DataFrame([code, keys, identifier, '']).T
                        add_l.columns = ['code', 'symbol', 'identifier', 'restricted']
                        legend = legend.append(add_l, ignore_index = True, sort = False)            
        try:
            if fundname != '':
                df = pd.read_html(str(i), flavor="bs4")
                for t in df:
                    count = count +1
                    tables.append(t)
                    add_legend = get_legend(t)
                    if len(add_legend.index) > 0:
                        add_legend.loc[:, 'identifier'] = fundname
                        if len(legend.index) > 0:
                            legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
                        else:
                            legend  = add_legend.copy()

                    cleaned_df, headername = get_holdings(t, headername)
                    if len(cleaned_df.index) > 1:
                        cleaned_df.loc[:, 'fund name'] = fundname
                        cleaned_df.loc[:, 'identifier'] = fundname
                        if len(holdings.index) > 0:
                            holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
                        else:
                            holdings = cleaned_df.copy()
        except: 
            pass


    
    holdings['value multiplier'] = ''
    holdings['textfile'] = text
    holdings['CIK'] = CIK
    holdings['date_filed'] = '' 
    holdings['company conformed name'] = ''
    holdings['reporting_date'] = ''

    try:
        date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
        conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
        reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
    except IndexError:
        date_filed = ''
        conformed_name = ''

    holdings.loc[:, 'date_filed'] = date_filed
    holdings.loc[:, 'company conformed name'] = conformed_name
    holdings.loc[:, 'reporting_date'] = reporting_date

    holdings.index = pd.RangeIndex(len(holdings.index))

    for z in weblink[CIK]:
        if text in z:
            matching_link = z

    holdings['weblink'] = matching_link
    legend['weblink'] = matching_link
    legend['textfile'] = text
    
    return holdings, legend




check panel1 for the sample file

In [67]:
#holdings, legend = getpanel1('36405', '0000932471-18-007518.txt', panel2)
holdings, legend = getpanel1('857489', '0000932471-06-000632.txt', panel2)

In [68]:
holdings.to_csv(os.path.join(output_directory, fund_name + '_test_holdings2.csv'), sep = ',')
legend.to_csv(os.path.join(output_directory, fund_name + '_test_legend2.csv'), sep = ',')
print(holdings)

    holdings shares holdings value  \
0            372488          26366   
1            790127          19127   
2            301944          16823   
3            760127           7830   
4             17406           7788   
5            142205           6271   
6             44741           5274   
7             22932           4433   
8            216721           4039   
9             20475           2353   
10            23387           1845   
11             9576           1436   
12            46417           1305   
13          2643524          91919   
14           413601          41510   
15          1237128          30390   
16           410656          19325   
17           156106          17471   
18           143284          16182   
19           367519          11324   
20           160093          11001   
21           197462           9748   
22            54518           7172   
23            35722           5075   
24            64364           4842   
25          

## Run on All CIKS

In [ ]:
panel1 = pd.DataFrame()


panel1 = pd.DataFrame()
panel1_legend = pd.DataFrame()
error_panel1 = []
no_holdings = {}

CIK_LIST = ['106444', '36405', '752177', '225997', '734383', '932471', '794105', '826473', '1273878', '857489', '1004655', '836906', '857490', '821404', '788599', '862341', '788606', '891190', '791107', '34066', '1070414', '105544', '105563', '68138', '52848', '799127', '736054', '106830', '313850', '107606']


for CIK in CIK_LIST:
    no_holdings[CIK] = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        if '.csv' not in i and i not in errors[CIK]:
            holdings, legend = getpanel1(str(CIK), str(i), panel2)
            print(holdings)
            if len(holdings) == 0:
                no_holdings[CIK].append(i)

            if panel1.empty:
                panel1 = holdings.copy()
                panel1_legend = legend.copy()
            else:
                panel1 = pd.concat([panel1, holdings], axis = 0, sort = False)
                panel1_legend = pd.concat([panel1_legend , legend], axis = 0, sort = False)

            panel1.index = pd.RangeIndex(len(panel1.index))


In [ ]:
holdings, legend = getpanel1(str(CIK), str(i), panel2)
str(CIK)


In [ ]:
panel1.loc[:,'acq name'] = ''
panel1.loc[:,'acq date'] = ''
panel1.loc[:,'acq cost'] = ''
print(output_directory)
panel1.to_csv(os.path.join(output_directory, fund_name + '_panel1.csv'), sep = ',')
panel1_legend.to_csv(os.path.join(output_directory, fund_name + '_panel1_legend.csv'), sep = ',')

In [ ]:
panel1['CIK'].unique()

In [ ]:

panel1_legend.loc[:,'restricted'] = 'no'

panel1_legend.loc[panel1_legend['code'].str.contains("restrict") | panel1_legend['code'].str.contains("level 3 security") \
              | panel1_legend['code'].str.contains("exempt from registration") |panel1_legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = panel1_legend.loc[panel1_legend['restricted'].str.contains("yes")]

restricted.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted_legend.csv'), sep = ',')
print(restricted)

panel1.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    try:
        panel1.loc[(panel1['identifier'] == row['identifier']) & (panel1['textfile'] == row.textfile) \
                       & (panel1['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    except:
        pass
    
restricted_holdings = panel1.loc[(panel1['restricted'] == 'yes')]
restricted_holdings.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted.csv'), sep = ',')

In [ ]:
restricted_holdings